# HPWREN Camera Metadata Process (Firemap API)

Summary: <br>
The HPWREN camera metadata is stored within the San Diego Supercomputer Center and is accessible via the FireMap API.

firemap api --> all cam metadata --> filter for hpwren only --> write to csv

Open Questions: <br>
- Are these active cameras only? (Are there ever historical cameras that are no longer active?)

## Get All Camera Metadata From Firemap API

In [1]:
import json
from datetime import datetime

import pandas as pd
import pytz
import requests

In [2]:
BASE_URL = "https://firemap.sdsc.edu/pylaski/"

In [3]:
# Get data for ALL cameras
# https://firemap.sdsc.edu/pylaski/stations?camera=only&selection=boundingBox&minLat=0&maxLat=90&minLon=-180&maxLon=0
entity = "stations"
payload = {
    "camera": "only",
    "selection": "boundingBox",
    "minLat": 0,
    "maxLat": 90,  # northern hemisphere
    "minLon": -180,
    "maxLon": 0,  # western hemisphere
}
response = requests.get(f"{BASE_URL}{entity}", params=payload)

In [4]:
if response.status_code in range(200, 300):  # successful response range
    response_json = response.json()
    features = response_json["features"]
else:
    # TODO:
    # Add error handling
    print("Error!")

In [5]:
cameras_df = pd.json_normalize(features)
print(f"Total cameras: {len(cameras_df['properties.description.id'])}")

Total cameras: 1138


In [6]:
# cameras_df

In [7]:
# Camera breakdown
# all camera metadata either has hpwren or axis prefix
cameras_df["properties.description.id"].str.split(r"_|-", expand=True)[0].str.extract(
    r"([a-zA-Z]+)"
)[0].str.lower().value_counts()

axis      1035
hpwren     103
Name: 0, dtype: int64

## Filter for HPWREN cameras only

In [8]:
# just get hpwren cameras (ignore axis)

hpwren_cameras_df = cameras_df[
    cameras_df["properties.description.id"]
    .str.lower()
    .str.contains("hpwren", regex=False)
].copy()

# dictoinary example below:
# hpwren_cameras = []
# for feat in features:
#     feat_id = feat["properties"]["description"]["id"]
#     if "hpwren" in feat_id.lower():
#         # print(feat_id)
#         hpwren_cameras.append(feat)

In [9]:
len(hpwren_cameras_df)

103

## Transformations

In [10]:
# Extract direction from description
hpwren_cameras_df["direction"] = hpwren_cameras_df[
    "properties.description.id"
].str.split("_", n=1, expand=True)[1]

In [11]:
hpwren_cameras_df["direction"].value_counts()

north                26
east                 26
south                25
west                 25
unknown direction     1
Name: direction, dtype: int64

## Write camera metadata to raw and processed folders

In [12]:
# current_time_str = datetime.now(tz=pytz.UTC).strftime("%m/%d/%Y, %H:%M:%S")
current_time_str = datetime.now(tz=pytz.UTC).strftime("%Y%m%d_%H_%M_%S")

# all cameras
raw_output_path = f"../../data/raw/camera_metadata.csv"
cameras_df.to_csv(raw_output_path, index=False)

# hpwren cameras only
processed_output_path = f"../../data/processed/camera_metadata_hpwren.csv"
hpwren_cameras_df.to_csv(processed_output_path, index=False)